In [23]:
import pandas as pd

# URL of the CSV
url = "https://datahub.io/core/pharmaceutical-drug-spending/_r/-/data/data.csv"

# Read into a DataFrame
df = pd.read_csv(url)

# Quick check
df.head()


,LOCATION,TIME,PC_HEALTHXP,PC_GDP,USD_CAP,TOTAL_SPEND
0,AUS,1971,15.992,0.726,33.990,439.73
1,AUS,1972,15.091,0.685,34.184,450.44
2,AUS,1973,15.117,0.681,37.956,507.85
3,AUS,1974,14.771,0.754,45.338,622.17
4,AUS,1975,11.849,0.682,44.363,616.34


In [24]:
# Step 1: Missing Values Check - confirms data completeness
print("Step 1: Missing Values per Column")
print(df.isnull().sum(), "\n")

Step 1: Missing Values per Column
LOCATION       0
TIME           0
PC_HEALTHXP    0
PC_GDP         0
USD_CAP        0
TOTAL_SPEND    0
dtype: int64 



In [25]:

# Step 2: Time Coverage Check - shows each country’s data span
coverage = df.groupby('LOCATION')['TIME'].agg(
    first_year='min',
    last_year='max',
    years_reported=lambda x: x.nunique()
).reset_index()
print("1. Time Coverage by Country:")
print(coverage.to_string(index=False))

1. Time Coverage by Country:
LOCATION  first_year  last_year  years_reported
     AUS        1971       2020              50
     AUT        1990       2021              32
     BEL        1970       2021              47
     BGR        2003       2021              16
     BRA        2015       2019               5
     CAN        1970       2022              53
     CHE        2010       2021              12
     CHL        2019       2021               3
     COL        2013       2017               5
     CRI        2011       2021              11
     CYP        2003       2021              19
     CZE        1990       2021              32
     DEU        1970       2021              51
     DNK        1980       2022              43
     ESP        1980       2021              38
     EST        1999       2021              23
     FIN        1970       2021              52
     FRA        1970       2021              36
     GBR        1970       2021              36
     GRC   

In [26]:
# Step 3: Summary Statistics - provides a quick check on each numeric variable 
summary = df[['PC_HEALTHXP', 'PC_GDP', 'USD_CAP', 'TOTAL_SPEND']].describe().T
print("\n3. Summary Statistics:")
print(summary.to_string())


3. Summary Statistics:
              count          mean           std    min      25%       50%       75%         max
PC_HEALTHXP  1341.0     16.632877      6.613320  5.545   11.849    15.238    20.580      40.239
PC_GDP       1341.0      1.215524      0.479568  0.207    0.802     1.187     1.524       2.870
USD_CAP      1341.0    338.278398    226.158326  3.628  144.592   317.795   496.858    1432.309
TOTAL_SPEND  1341.0  14439.322088  43259.358895  5.810  756.280  2786.190  8794.360  475596.740


In [27]:
# Step 4: Outlier Detection (1st & 99th percentile)
usd_low, usd_high = df['USD_CAP'].quantile([0.01, 0.99])
ts_low, ts_high = df['TOTAL_SPEND'].quantile([0.01, 0.99])
outliers = df[
    (df['USD_CAP'] < usd_low) | (df['USD_CAP'] > usd_high) |
    (df['TOTAL_SPEND'] < ts_low) | (df['TOTAL_SPEND'] > ts_high)
][['LOCATION', 'TIME', 'USD_CAP', 'TOTAL_SPEND']]
print("\n4. Outliers (1st & 99th percentile):")
print(outliers.to_string(index=False))


4. Outliers (1st & 99th percentile):
LOCATION  TIME  USD_CAP  TOTAL_SPEND
     DEU  2021 1042.460     86728.58
     ISL  1970   28.437         5.81
     ISL  1971   38.402         7.91
     ISL  1972   43.415         9.08
     ISL  1973   42.803         9.09
     ISL  1974   46.620        10.03
     ISL  1975   58.281        12.70
     ISL  1976   56.601        12.46
     ISL  1977   65.760        14.59
     ISL  1978   80.924        18.09
     ISL  1979   92.839        20.96
     ISL  1980  120.025        27.38
     KOR  1970    3.976       128.19
     KOR  1971    3.628       119.30
     KOR  1972    3.754       125.78
     KOR  1973    3.853       131.40
     KOR  1974    3.826       132.73
     KOR  1975    6.505       229.50
     KOR  1976    7.968       285.64
     KOR  1977    8.546       311.18
     LUX  1976   60.128        21.69
     LUX  1977   63.006        22.77
     LUX  1978   74.606        27.01
     PRT  1970    6.345        55.08
     TUR  1981    9.685       448.62


In [28]:
# Step 5: Correlation Matrix
corr = df[['PC_HEALTHXP', 'PC_GDP', 'USD_CAP', 'TOTAL_SPEND']].corr()
print("\n5. Correlation Matrix:")
print(corr.to_string())


5. Correlation Matrix:
             PC_HEALTHXP    PC_GDP   USD_CAP  TOTAL_SPEND
PC_HEALTHXP     1.000000  0.718099  0.112574    -0.055372
PC_GDP          0.718099  1.000000  0.643091     0.307392
USD_CAP         0.112574  0.643091  1.000000     0.521185
TOTAL_SPEND    -0.055372  0.307392  0.521185     1.000000


In [29]:
# Step 6: Feasibility Check (>=10 years of data)
feasible = coverage[coverage['years_reported'] >= 10]
print("\n6. Countries with ≥10 Years of Data:")
print(feasible.to_string(index=False))


6. Countries with ≥10 Years of Data:
LOCATION  first_year  last_year  years_reported
     AUS        1971       2020              50
     AUT        1990       2021              32
     BEL        1970       2021              47
     BGR        2003       2021              16
     CAN        1970       2022              53
     CHE        2010       2021              12
     CRI        2011       2021              11
     CYP        2003       2021              19
     CZE        1990       2021              32
     DEU        1970       2021              51
     DNK        1980       2022              43
     ESP        1980       2021              38
     EST        1999       2021              23
     FIN        1970       2021              52
     FRA        1970       2021              36
     GBR        1970       2021              36
     GRC        1988       2021              34
     HRV        2011       2021              11
     HUN        1991       2021              28
  

In [30]:
# Exclude countries with less than 10 years of data
df_filtered = df.groupby('LOCATION').filter(lambda x: x['TIME'].nunique() >= 10)

# check
print(f"Original dataset rows: {len(df)}")
print(f"Filtered dataset rows: {len(df_filtered)}")
print("Countries retained:", df_filtered['LOCATION'].unique())

#Identify and display excluded countries
excluded = coverage[coverage['years_reported'] < 10]
print("Excluded Countries (fewer than 10 years of data):")
print(excluded.to_string(index=False))

Original dataset rows: 1341
Filtered dataset rows: 1321
Countries retained: ['AUS' 'AUT' 'BEL' 'BGR' 'CAN' 'CHE' 'CRI' 'CYP' 'CZE' 'DEU' 'DNK' 'ESP'
 'EST' 'FIN' 'FRA' 'GBR' 'GRC' 'HRV' 'HUN' 'IRL' 'ISL' 'ISR' 'ITA' 'JPN'
 'KOR' 'LTU' 'LUX' 'LVA' 'MEX' 'NLD' 'NOR' 'NZL' 'POL' 'PRT' 'ROU' 'SVK'
 'SVN' 'SWE' 'TUR' 'USA']
Excluded Countries (fewer than 10 years of data):
LOCATION  first_year  last_year  years_reported
     BRA        2015       2019               5
     CHL        2019       2021               3
     COL        2013       2017               5
     MLT        2014       2020               7


In [31]:
# Compute number of distinct years per country
years_per_country = df.groupby('LOCATION')['TIME'].nunique()

# Identify countries with 10+ years of data
countries_10plus = years_per_country[years_per_country >= 10].index.tolist()

# Create filtered DataFrame
df_10plus = df[df['LOCATION'].isin(countries_10plus)].copy()

# Check results
print(f"Filtered DataFrame shape: {df_10plus.shape}")
print("Countries included:", sorted(df_10plus['LOCATION'].unique()))

Filtered DataFrame shape: (1321, 6)
Countries included: ['AUS', 'AUT', 'BEL', 'BGR', 'CAN', 'CHE', 'CRI', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN', 'IRL', 'ISL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'SWE', 'TUR', 'USA']


In [32]:
print(df_10plus)

     LOCATION  TIME  PC_HEALTHXP  PC_GDP   USD_CAP  TOTAL_SPEND
0         AUS  1971       15.992   0.726    33.990       439.73
1         AUS  1972       15.091   0.685    34.184       450.44
2         AUS  1973       15.117   0.681    37.956       507.85
3         AUS  1974       14.771   0.754    45.338       622.17
4         AUS  1975       11.849   0.682    44.363       616.34
...       ...   ...          ...     ...       ...          ...
1336      USA  2017       11.954   2.004  1200.769    390396.57
1337      USA  2018       11.766   1.957  1229.266    401771.09
1338      USA  2019       11.767   1.961  1277.032    419287.86
1339      USA  2020       11.037   2.070  1315.218    436030.19
1340      USA  2021       11.743   2.039  1432.309    475596.74

[1321 rows x 6 columns]


In [33]:
# Load population data from DataHub
pop_url = "https://datahub.io/core/population/_r/-/data/population.csv"
pop = pd.read_csv(pop_url)

In [34]:
# Inspect columns (uncomment to verify)
print(pop.columns)

Index(['Country Name', 'Country Code', 'Year', 'Value'], dtype='object')


In [35]:
#Rename to match df_10plus
pop = pop.rename(columns={
    "Country Code": "LOCATION",
    "Year": "TIME",
    "Value": "POPULATION"
})

In [36]:
df_with_pop = df_10plus.merge(
    pop[["LOCATION", "TIME", "POPULATION"]],
    on=["LOCATION", "TIME"],
    how="left"
)

In [37]:
#Check for missing population entries
missing_count = df_with_pop["POPULATION"].isnull().sum()
print(f"Rows missing population after merge: {missing_count}")

Rows missing population after merge: 0


In [38]:
#Preview merged DataFrame
print(df_with_pop.head())

  LOCATION  TIME  PC_HEALTHXP  PC_GDP  USD_CAP  TOTAL_SPEND  POPULATION
0      AUS  1971       15.992   0.726   33.990       439.73  12937000.0
1      AUS  1972       15.091   0.685   34.184       450.44  13177000.0
2      AUS  1973       15.117   0.681   37.956       507.85  13380000.0
3      AUS  1974       14.771   0.754   45.338       622.17  13723000.0
4      AUS  1975       11.849   0.682   44.363       616.34  13893000.0


In [39]:
# Correlation Matrix
corr = df_with_pop[['PC_HEALTHXP', 'PC_GDP', 'USD_CAP', 'TOTAL_SPEND',"POPULATION"]].corr()
print("Correlation Matrix:")
print(corr.to_string())

Correlation Matrix:
             PC_HEALTHXP    PC_GDP   USD_CAP  TOTAL_SPEND  POPULATION
PC_HEALTHXP     1.000000  0.719599  0.110652    -0.056234   -0.017329
PC_GDP          0.719599  1.000000  0.643487     0.307879    0.232212
USD_CAP         0.110652  0.643487  1.000000     0.529583    0.320995
TOTAL_SPEND    -0.056234  0.307879  0.529583     1.000000    0.845141
POPULATION     -0.017329  0.232212  0.320995     0.845141    1.000000


In [40]:
# Compute the number of unique countries reporting per year
country_counts = df_with_pop.groupby('TIME')['LOCATION'].nunique().rename('country_count')

# Sort years by descending count
counts_sorted = country_counts.sort_values(ascending=False)

# Identify the year(s) with the maximum number of reporting countries
max_count = counts_sorted.max()
years_most_reported = counts_sorted[counts_sorted == max_count].index.tolist()

# Display the sorted counts and the top reporting year(s)
print("Years sorted by number of countries reporting:")
print(counts_sorted.to_string())
print(f"\nMaximum number of countries reporting: {max_count}")
print(f"Year(s) with most countries reporting: {years_most_reported}")

Years sorted by number of countries reporting:
TIME
2019    38
2018    38
2017    38
2016    38
2015    38
2013    38
2014    38
2020    38
2012    37
2011    36
2006    35
2021    35
2007    35
2008    34
2005    34
2004    34
2010    34
2009    33
2003    31
2002    27
1990    25
2000    25
1996    25
1994    25
2001    25
1995    25
1997    24
1999    24
1992    23
1991    23
1993    23
1989    21
1988    21
1998    21
1985    20
1987    20
1984    19
1980    19
1986    19
1981    18
1983    18
1982    18
1978    14
1976    14
1977    14
1979    14
1975    13
1971    11
1970    11
1974    11
1972    11
1973    11
2022     7

Maximum number of countries reporting: 38
Year(s) with most countries reporting: [2019, 2018, 2017, 2016, 2015, 2013, 2014, 2020]


In [41]:
# Filter to the years 2011 through 2020 inclusive
df_2011_2020 = df_with_pop[(df_with_pop['TIME'] >= 2011) & (df_with_pop['TIME'] <= 2020)].copy()

# Verify the range and size
print(f"Year range in filtered DataFrame: {df_2011_2020['TIME'].min()} to {df_2011_2020['TIME'].max()}")
print(f"Filtered DataFrame shape: {df_2011_2020.shape}")

# List unique years to confirm
print("Unique years:", sorted(df_2011_2020['TIME'].unique()))
print("Countries included:",(df_2011_2020['LOCATION'].unique()))

Year range in filtered DataFrame: 2011 to 2020
Filtered DataFrame shape: (377, 7)
Unique years: [np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020)]
Countries included: ['AUS' 'AUT' 'BEL' 'BGR' 'CAN' 'CHE' 'CRI' 'CYP' 'CZE' 'DEU' 'DNK' 'ESP'
 'EST' 'FIN' 'FRA' 'GBR' 'GRC' 'HRV' 'HUN' 'IRL' 'ISL' 'ISR' 'ITA' 'JPN'
 'KOR' 'LTU' 'LUX' 'LVA' 'MEX' 'NLD' 'NOR' 'POL' 'PRT' 'ROU' 'SVK' 'SVN'
 'SWE' 'USA']


In [42]:
# Save the df_2011_2020 DataFrame to a CSV on your local machine
#output_path = r'C:\Users\alebe\DSI\Pharma_Spending_Pattern\Cleaned_data_with_population.csv'
#df_2011_2020.to_csv(output_path, index=False)

df = df_2011_2020

# Sort by LOCATION and TIME for correct growth calculation
df.sort_values(['LOCATION', 'TIME'], inplace=True)
# Calculate growth rates for selected columns
growth_columns = ['PC_HEALTHXP', 'PC_GDP', 'USD_CAP','TOTAL_SPEND','POPULATION']
for col in growth_columns:
    df[f'{col}_GROWTH'] = df.groupby('LOCATION')[col].pct_change()

# Identify the column with the highest growth each year for each location
df['HIGHEST_GROWTH_METRIC'] = df[[f'{col}_GROWTH' for col in growth_columns]].idxmax(axis=1)
df['HIGHEST_GROWTH_VALUE'] = df[[f'{col}_GROWTH' for col in growth_columns]].max(axis=1)

# Optional: clean the metric name (remove "_GROWTH" suffix)
df['HIGHEST_GROWTH_METRIC'] = df['HIGHEST_GROWTH_METRIC'].str.replace('_GROWTH', '')

# Save the result to csv
df.to_csv('Cleaned_data_with_population.csv', index=False)


C:\Users\bmanna\AppData\Local\Temp\ipykernel_10528\473226752.py:15: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df['HIGHEST_GROWTH_METRIC'] = df[[f'{col}_GROWTH' for col in growth_columns]].idxmax(axis=1)
